In [ ]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
MAIN_FOLDER = main_folder("eastindiacompany")

create_database('blp_dataset')
create_table('products_tmp_eic')

In [ ]:
folder = Path(f"{MAIN_FOLDER}")

for folders in os.listdir(folder):
    folder_main = Path(f"{MAIN_FOLDER}\{folders}\listings")
    
    for item in os.listdir(folder_main):
        print(Path(f"{folder_main}\{item}"))

        page_content = BeautifulSoup(open(Path(f"{folder_main}\{item}"), encoding='utf8').read())

        if page_content is not None:
            ## GET DESCRIPTION
            description = ""
            desc_content = page_content.find("div", attrs={"class": "tab-content-section"})

            if desc_content is not None:
                description = get_content(desc_content)

            ## GET TITLE
            title = ""
            title_content = page_content.find("h2")

            if title_content is not None:
                title = get_content(title_content)

            ## GET SELLER
            seller = ""
            seller_content = page_content.find("p", attrs={"class": "by-merchant"})

            if seller_content is not None:
                raw_seller = seller_content.find("a")

                if raw_seller is not None:
                    seller = get_content(raw_seller)

            if description != "" or title != "":
                # Insert product into database
                query = "INSERT INTO products_tmp_eic (name, description, market_name, seller_name, illegal) VALUES (%s, %s, %s, %s, %s)"
                values = (title, description, "East India Company", seller, "t")

                cursor.execute(query, values)
                conn.commit()

                product_id = cursor.lastrowid

                ## GET IMAGE
                img_content = page_content.find("div", attrs={"class": "image-frame-full"})

                if img_content is not None:
                    img_tag = img_content.find("img")

                    if img_tag is not None:
                        img = img_tag["src"]

                        if img is not None:
                            try:
                                data = "" + img.replace("data:image/jpg;base64,", "")
                                data = "" + data.replace("data:image/jpeg;base64,", "")
                                data = "" + data.replace("data:image/png;base64,", "")
                                imgdata = base64.b64decode(data)

                                img_filename = Path(f"D:\images\_eic\{product_id}.jpeg")

                                with open(img_filename, 'wb') as f:
                                    f.write(imgdata)
                                    f.close
                            except:
                                print(f"Image {product_id} has error. Except")